In [38]:
import json
import pandas as pd

In [2]:
with open('match_stats.json', 'r') as f:
    df = pd.DataFrame.from_dict(pd.json_normalize(json.loads(f.read())))

Добавим признак, который будет показывать историю личных встреч из нашего датасета. Для этого введем признак head-to-head, который будет считать сумму результатов матчей (за период до прошедшего матча) между командами по следующему принципу: +1 за победу, 0 за ничью и -1 за проигрыш.

In [35]:
teams = df['home.team'].unique()

In [36]:
def head_to_head(team1: str, team2: str, df: pd.DataFrame) -> float:
    home_matches = df[(df['home.team'] == team1) & (df['guest.team'] == team2)]
    score = 0
    score += (home_matches['home.score'] > home_matches['guest.score']).sum()
    score -= (home_matches['home.score'] < home_matches['guest.score']).sum()
    guest_matches = df[(df['home.team'] == team2) & (df['guest.team'] == team1)]
    score -= (guest_matches['home.score'] > guest_matches['guest.score']).sum()
    score += (guest_matches['home.score'] < guest_matches['guest.score']).sum()
    return score

In [37]:
df['head-to-head'] = df.apply(lambda row: head_to_head(row['home.team'], row['guest.team'], df[df['match_date'] < row['match_date']]), axis=1)

In [39]:
df.head()

,match_date,match_week,home.team,home.score,home.xG,home.possession,home.passing_accuracy,home.shots_on_target,home.yellow_cards_count,home.red_cards_count,...,guest.touches,guest.tackles,guest.interceptions,guest.aerials_won,guest.clearances,guest.offsides,guest.goal_kicks,guest.throw_ins,guest.long_balls,head-to-head
0,2022-08-05,1,Crystal Palace,0,1.2,0.56,0.84,0.20,1,0,...,599.0,29.0,9,14.0,24.0,2.0,2.0,14.0,59.0,-2
1,2022-08-06,1,Fulham,2,1.2,0.33,0.60,0.25,2,0,...,784.0,11.0,10,13.0,16.0,4.0,5.0,35.0,94.0,-1
2,2022-08-06,1,Tottenham Hotspur,4,1.5,0.58,0.83,0.44,3,0,...,554.0,14.0,13,11.0,21.0,0.0,4.0,14.0,61.0,4
3,2022-08-06,1,Newcastle United,2,1.7,0.61,0.79,0.39,0,0,...,475.0,15.0,10,16.0,37.0,0.0,12.0,26.0,70.0,0
4,2022-08-06,1,Leeds United,2,0.8,0.40,0.74,0.33,2,0,...,720.0,16.0,14,7.0,17.0,1.0,9.0,23.0,72.0,-1
